In [ ]:
!apt-get install tesseract-ocr -y
!pip install pytesseract
!apt-get install poppler-utils -y
!pip install pdf2image
!apt-get install tesseract-ocr-swe
!pip install PyMuPDF jiwer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,935 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re
import os
import glob

def clean_text(text):
    """
    Remove invalid characters from the text.
    """
    invalid_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_chars.sub('', text)

def process_pdf(pdf_path):
    # Generate TXT filename by replacing the PDF extension with TXT
    txt_file = os.path.splitext(pdf_path)[0] + '.txt'

    try:
        # Convert PDF to list of images
        images = convert_from_path(pdf_path)
    except Exception as e:
        print(f"Failed to convert {pdf_path} to images: {e}")
        return

    try:
        with open(txt_file, 'w', encoding='utf-8') as file:
            for i, image in enumerate(images):
                # Perform OCR
                text = pytesseract.image_to_string(image, lang='swe')

                # Clean the OCR text
                clean_text_data = clean_text(text)

                # Write each page's text to the TXT file
                file.write(f"Page {i+1}\n{clean_text_data}\n")
                file.write("\n")  # Add a page break in the text file
    except Exception as e:
        print(f"Error writing to file {txt_file}: {e}")
    else:
        print(f"The text file {txt_file} has been created successfully.")

directory = 'table/'
pdf_files = glob.glob(os.path.join(directory, '*.pdf'))
for pdf_file in pdf_files:
    process_pdf(pdf_file)


The text file table/BZO3QEJ3CE2OQFGHA7ZD3CDOWHI543UM.txt has been created successfully.
The text file table/ADKCW3EY3SXNVTZUASDS7GAKEJ4GIQBI.txt has been created successfully.
The text file table/6G5OZCK6EQTZ35IQ7FV4UW64WVMWAW4T.txt has been created successfully.
The text file table/47HBHTDSTAKCJIYQ22345FHWVNVJMXJL.txt has been created successfully.
The text file table/46QMULSFORX2CQ75FYZAEACXSD7AJKPG.txt has been created successfully.
The text file table/AYZVGVPYWC6TNBDO76IU7PON3RZ22WQP.txt has been created successfully.
The text file table/6REZDVWAZINCN3EC63464SLUVI2QJHJW.txt has been created successfully.
The text file table/BYAGGYJAJF54MOS44JDLAZJZ2CJNQXNH.txt has been created successfully.
The text file table/7WOCAN6T6B5JRJB4C5LAIHLE6ERU2V22.txt has been created successfully.
The text file table/AB2LVVUR7Q4T7O5QDSUALJSM5UMJH2HT.txt has been created successfully.
The text file table/7GB7EXTYK2SHE3R3CBCOYKLOQT4CMEAF.txt has been created successfully.
The text file table/76P47DRTKG4I

In [ ]:
!unzip pic-filename.zip

In [ ]:
!zip -r table-filename.zip table

In [ ]:
import pytesseract
from pdf2image import convert_from_path
import re
import os
from jiwer import wer, cer

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

def process_file_pair(pdf_path, ground_truth_path):
    """
    Convert PDF to images, perform OCR, and calculate WER and CER against ground truth text.
    """
    try:
        images = convert_from_path(pdf_path)
        all_text = []

        for image in images:
            text = pytesseract.image_to_string(image, lang='swe')
            clean_text_data = clean_text(text)
            all_text.append(clean_text_data)

        ocr_output_text = '\n'.join(all_text)

        with open(ground_truth_path, 'r', encoding='utf-8') as gt_file:
            ground_truth_text = gt_file.read()

        calculated_wer = wer(ground_truth_text, ocr_output_text)
        calculated_cer = cer(ground_truth_text, ocr_output_text)

        print(f"Processing {pdf_path}...")
        print(f"WER: {calculated_wer}")
        print(f"CER: {calculated_cer}")
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")

def get_file_pairs(directory):
    """
    Returns a list of (pdf_path, txt_path) tuples from the directory.
    """
    pdf_files = [f for f in os.listdir(directory) if f.lower().endswith('.pdf')]
    file_pairs = [(os.path.join(directory, f), os.path.join(directory, f.replace('.pdf', '.txt')))
                  for f in pdf_files if os.path.exists(os.path.join(directory, f.replace('.pdf', '.txt')))]
    return file_pairs

def main():
    directory_path = 'text/'
    file_pairs = get_file_pairs(directory_path)

    if not file_pairs:
        print("No valid file pairs found.")
        return

    for pdf_path, txt_path in file_pairs:
        process_file_pair(pdf_path, txt_path)

if __name__ == "__main__":
    main()


Processing text/2EDEPZ4VHTLPTWSZR6FAVUJ3B2ZVSIPS.pdf...
WER: 0.04854368932038835
CER: 0.03259452411994785


In [ ]:
#Extracting metadata from the PDF based on keywords
import pytesseract
from pdf2image import convert_from_path
import xml.etree.ElementTree as ET
import re
import os
import cv2
import numpy as np

# Function to remove invalid XML characters
def clean_text(text):
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

def preprocess_image(image):
    """Apply image preprocessing steps to improve OCR accuracy."""
    # Convert image to grayscale
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # Apply thresholding to binarize the image
    _, thresh_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Optionally apply dilation to close gaps between letters; adjust kernel size as needed
    kernel = np.ones((2,2), np.uint8)
    dilated_image = cv2.dilate(thresh_image, kernel, iterations=1)
    return dilated_image

def print_first_keywords_and_values(xml_root, keywords):
    """Print the first occurrence of keywords and their values from an XML root."""
    # Initialize a set to track which keywords have been printed
    printed_keywords = set()

    for item in xml_root.findall('item'):
        item_text = item.text or ''
        for keyword in keywords:
            # Only proceed if the keyword hasn't been printed yet
            if keyword not in printed_keywords and keyword.lower() in item_text.lower():
                # Extract the value and print
                value = item_text.split(keyword)[-1].strip()
                print(f"{keyword}: {value}")
                # Add the keyword to the set of printed keywords
                printed_keywords.add(keyword)
                break  # Break the loop after printing the first occurrence

pdf_path = '3.pdf'
xml_file = os.path.splitext(pdf_path)[0] + '_output.xml'

# Convert PDF pages to images
images = convert_from_path(pdf_path)

# Perform OCR on each image and save the text in the XML structure
root = ET.Element("root")
for image in images:
    processed_image = preprocess_image(image)
    text = pytesseract.image_to_string(processed_image, lang='swe', config='--psm 6')
    clean_text_data = clean_text(text)

    # Split the cleaned text into lines and then add each line as an item to the XML root
    for line in clean_text_data.split('\n'):
        if line.strip():  # Skip empty lines
            item = ET.SubElement(root, "item")
            item.text = line.strip()

# Write the XML to a file
tree = ET.ElementTree(root)
tree.write(xml_file, encoding="utf-8", xml_declaration=True)
print(f"The OCR process is complete, see the generated XML file: {xml_file}")

# Define the keywords to search for
keywords = [
    "Fastighetsägare", "Namn", "Adress", "Postnr och ort",
    "Fastighetsbeteckning", "Anläggare av ledning"
]

# Parse the XML file just created and search for the keywords
try:
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Print the first occurrence of keywords and values
    print_first_keywords_and_values(root, keywords)

except Exception as e:
    print(f"Error during XML parsing: {e}")


The OCR process is complete, see the generated XML file: 3_output.xml
Adress: —
Postnr och ort: 
Fastighetsbeteckning: : NOSIo. 4 L
Anläggare av ledning: Norsjö kommun
Fastighetsägare: n ger anläggaren rätt till markutrymme på angiven fastighet för anläggning,
Namn: förtydligande (texta) För Norsjö kommun


In [ ]:
#Calculate the time duration

import pytesseract
from pdf2image import convert_from_path
import xml.etree.ElementTree as ET
import re
import os
import cv2
import numpy as np
import time  # Import the time module

# Function to remove invalid XML characters
def clean_text(text):
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

def preprocess_image(image):
    """Apply image preprocessing steps to improve OCR accuracy."""
    # Convert image to grayscale
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # Apply thresholding to binarize the image
    _, thresh_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Optionally apply dilation to close gaps between letters; adjust kernel size as needed
    kernel = np.ones((2,2), np.uint8)
    dilated_image = cv2.dilate(thresh_image, kernel, iterations=1)
    return dilated_image

def print_first_keywords_and_values(xml_root, keywords):
    """Print the first occurrence of keywords and their values from an XML root."""
    printed_keywords = set()
    for item in xml_root.findall('item'):
        item_text = item.text or ''
        for keyword in keywords:
            if keyword not in printed_keywords and keyword.lower() in item_text.lower():
                value = item_text.split(keyword)[-1].strip()
                print(f"{keyword}: {value}")
                printed_keywords.add(keyword)
                break

start_time = time.time()  # Start the timing

pdf_path = '4.pdf'
xml_file = os.path.splitext(pdf_path)[0] + '_output.xml'

# Convert PDF pages to images
images = convert_from_path(pdf_path)

# Perform OCR on each image and save the text in the XML structure
root = ET.Element("root")
for image in images:
    processed_image = preprocess_image(image)
    text = pytesseract.image_to_string(processed_image, lang='swe', config='--psm 6')
    clean_text_data = clean_text(text)
    for line in clean_text_data.split('\n'):
        if line.strip():
            item = ET.SubElement(root, "item")
            item.text = line.strip()

tree = ET.ElementTree(root)
tree.write(xml_file, encoding="utf-8", xml_declaration=True)
print(f"The OCR process is complete, see the generated XML file: {xml_file}")

keywords = [
    "Fastighetsägare", "Namn", "Adress", "Postnr och ort",
    "Fastighetsbeteckning", "Anläggare av ledning"
]

try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
    print_first_keywords_and_values(root, keywords)
except Exception as e:
    print(f"Error during XML parsing: {e}")

end_time = time.time()  # End the timing
elapsed_time = end_time - start_time
print(f"Total processing time: {elapsed_time} seconds")


The OCR process is complete, see the generated XML file: 4_output.xml
Adress: —
Postnr och ort: 
Fastighetsbeteckning: : NOSIo. 4 L
Anläggare av ledning: Norsjö kommun
Fastighetsägare: n ger anläggaren rätt till markutrymme på angiven fastighet för anläggning,
Namn: förtydligande (texta) För Norsjö kommun
Total processing time: 6.259115934371948 seconds
